In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from model import Model_1,Model_2,Model_3,Model_4,Model_5,Model_6,Model_7,Model_8,Model_9,Model_10

In [2]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [3]:
train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [4]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(



**Model_1**

In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_1().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
            Conv2d-3           [-1, 64, 24, 24]          18,496
              ReLU-4           [-1, 64, 24, 24]               0
            Conv2d-5          [-1, 128, 22, 22]          73,856
              ReLU-6          [-1, 128, 22, 22]               0
            Conv2d-7          [-1, 256, 20, 20]         295,168
              ReLU-8          [-1, 256, 20, 20]               0
         MaxPool2d-9          [-1, 256, 10, 10]               0
           Conv2d-10           [-1, 32, 10, 10]           8,224
           Conv2d-11             [-1, 64, 8, 8]          18,496
             ReLU-12             [-1, 64, 8, 8]               0
           Conv2d-13            [-1, 128, 6, 6]          73,856
             ReLU-14            [-

In [7]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [7]:
#model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.16863952577114105 Batch_id=468 Accuracy=48.77: 100%|██████████| 469/469 [00:23<00:00, 20.10it/s]



Test set: Average loss: 0.1309, Accuracy: 9590/10000 (95.90%)

EPOCH: 1


Loss=0.052918095141649246 Batch_id=468 Accuracy=96.93: 100%|██████████| 469/469 [00:26<00:00, 17.46it/s]



Test set: Average loss: 0.0658, Accuracy: 9791/10000 (97.91%)

EPOCH: 2


...


Test set: Average loss: 0.0321, Accuracy: 9911/10000 (99.11%)

EPOCH: 13


Loss=0.002737918868660927 Batch_id=468 Accuracy=99.82: 100%|██████████| 469/469 [00:22<00:00, 20.92it/s]



Test set: Average loss: 0.0355, Accuracy: 9916/10000 (99.16%)

EPOCH: 14


Loss=0.0014021910028532147 Batch_id=468 Accuracy=99.85: 100%|██████████| 469/469 [00:23<00:00, 19.77it/s]



Test set: Average loss: 0.0440, Accuracy: 9889/10000 (98.89%)



In [8]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_2().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
       BatchNorm2d-3           [-1, 32, 26, 26]              64
            Conv2d-4           [-1, 64, 24, 24]          18,496
              ReLU-5           [-1, 64, 24, 24]               0
       BatchNorm2d-6           [-1, 64, 24, 24]             128
            Conv2d-7          [-1, 128, 22, 22]          73,856
              ReLU-8          [-1, 128, 22, 22]               0
       BatchNorm2d-9          [-1, 128, 22, 22]             256
           Conv2d-10          [-1, 256, 20, 20]         295,168
             ReLU-11          [-1, 256, 20, 20]               0
      BatchNorm2d-12          [-1, 256, 20, 20]             512
        MaxPool2d-13          [-1, 256, 10, 10]               0
           Conv2d-14           [-1

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loss=0.021650319918990135 Batch_id=468 Accuracy=97.14: 100%|██████████| 469/469 [00:23<00:00, 20.14it/s]



Test set: Average loss: 0.0315, Accuracy: 9900/10000 (99.00%)

EPOCH: 1


Loss=0.05013532564043999 Batch_id=468 Accuracy=99.17: 100%|██████████| 469/469 [00:23<00:00, 19.72it/s]



Test set: Average loss: 0.0268, Accuracy: 9912/10000 (99.12%)

EPOCH: 2


...


Test set: Average loss: 0.0156, Accuracy: 9957/10000 (99.57%)

EPOCH: 13


Loss=0.0004852490674238652 Batch_id=468 Accuracy=100.00: 100%|██████████| 469/469 [00:23<00:00, 20.01it/s]



Test set: Average loss: 0.0151, Accuracy: 9956/10000 (99.56%)

EPOCH: 14


Loss=0.0003619023773353547 Batch_id=468 Accuracy=100.00: 100%|██████████| 469/469 [00:23<00:00, 19.65it/s]



Test set: Average loss: 0.0157, Accuracy: 9958/10000 (99.58%)



In [10]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_3().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
            Conv2d-4           [-1, 12, 24, 24]             876
              ReLU-5           [-1, 12, 24, 24]               0
       BatchNorm2d-6           [-1, 12, 24, 24]              24
         MaxPool2d-7           [-1, 12, 12, 12]               0
            Conv2d-8           [-1, 14, 10, 10]           1,526
              ReLU-9           [-1, 14, 10, 10]               0
      BatchNorm2d-10           [-1, 14, 10, 10]              28
           Conv2d-11            [-1, 8, 10, 10]             120
           Conv2d-12             [-1, 12, 8, 8]             876
             ReLU-13             [-1, 12, 8, 8]               0
      BatchNorm2d-14             [

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.15751191973686218 Batch_id=468 Accuracy=93.93: 100%|██████████| 469/469 [00:19<00:00, 23.80it/s]



Test set: Average loss: 0.0508, Accuracy: 9855/10000 (98.55%)

EPOCH: 1


Loss=0.04834945872426033 Batch_id=468 Accuracy=98.42: 100%|██████████| 469/469 [00:20<00:00, 22.73it/s]



Test set: Average loss: 0.0394, Accuracy: 9876/10000 (98.76%)

EPOCH: 2


...


Test set: Average loss: 0.0301, Accuracy: 9899/10000 (98.99%)

EPOCH: 13


Loss=0.021857842803001404 Batch_id=468 Accuracy=99.69: 100%|██████████| 469/469 [00:19<00:00, 23.67it/s]



Test set: Average loss: 0.0314, Accuracy: 9906/10000 (99.06%)

EPOCH: 14


Loss=0.011312375776469707 Batch_id=468 Accuracy=99.76: 100%|██████████| 469/469 [00:21<00:00, 22.18it/s]



Test set: Average loss: 0.0259, Accuracy: 9908/10000 (99.08%)



In [12]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_4().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
           Dropout-8           [-1, 12, 24, 24]               0
         MaxPool2d-9           [-1, 12, 12, 12]               0
           Conv2d-10           [-1, 14, 10, 10]           1,526
             ReLU-11           [-1, 14, 10, 10]               0
      BatchNorm2d-12           [-1, 14, 10, 10]              28
          Dropout-13           [-1, 14, 10, 10]               0
           Conv2d-14            [-

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.07587466388940811 Batch_id=468 Accuracy=93.42: 100%|██████████| 469/469 [00:22<00:00, 20.83it/s]



Test set: Average loss: 0.0542, Accuracy: 9837/10000 (98.37%)

EPOCH: 1


Loss=0.12082294374704361 Batch_id=468 Accuracy=98.00: 100%|██████████| 469/469 [00:19<00:00, 23.84it/s]



Test set: Average loss: 0.0413, Accuracy: 9855/10000 (98.55%)

EPOCH: 2


...


Test set: Average loss: 0.0199, Accuracy: 9928/10000 (99.28%)

EPOCH: 13


Loss=0.038419775664806366 Batch_id=468 Accuracy=99.23: 100%|██████████| 469/469 [00:20<00:00, 23.42it/s]



Test set: Average loss: 0.0217, Accuracy: 9931/10000 (99.31%)

EPOCH: 14


Loss=0.05103001371026039 Batch_id=468 Accuracy=99.21: 100%|██████████| 469/469 [00:19<00:00, 24.10it/s]



Test set: Average loss: 0.0199, Accuracy: 9938/10000 (99.38%)



In [14]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_5().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
           Dropout-8           [-1, 12, 24, 24]               0
         MaxPool2d-9           [-1, 12, 12, 12]               0
           Conv2d-10           [-1, 14, 10, 10]           1,526
             ReLU-11           [-1, 14, 10, 10]               0
      BatchNorm2d-12           [-1, 14, 10, 10]              28
          Dropout-13           [-1, 14, 10, 10]               0
           Conv2d-14            [-

In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.19020627439022064 Batch_id=468 Accuracy=85.61: 100%|██████████| 469/469 [00:20<00:00, 22.86it/s]



Test set: Average loss: 0.1780, Accuracy: 9436/10000 (94.36%)

EPOCH: 1


Loss=0.1974436640739441 Batch_id=468 Accuracy=95.16: 100%|██████████| 469/469 [00:21<00:00, 22.23it/s]



Test set: Average loss: 0.1433, Accuracy: 9524/10000 (95.24%)

EPOCH: 2


...


Test set: Average loss: 0.0628, Accuracy: 9788/10000 (97.88%)

EPOCH: 13


Loss=0.020556412637233734 Batch_id=468 Accuracy=97.62: 100%|██████████| 469/469 [00:21<00:00, 22.08it/s]



Test set: Average loss: 0.0667, Accuracy: 9780/10000 (97.80%)

EPOCH: 14


Loss=0.14640948176383972 Batch_id=468 Accuracy=97.70: 100%|██████████| 469/469 [00:19<00:00, 23.57it/s]



Test set: Average loss: 0.0522, Accuracy: 9828/10000 (98.28%)



In [16]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_6().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
         MaxPool2d-8           [-1, 12, 12, 12]               0
            Conv2d-9           [-1, 14, 10, 10]           1,526
             ReLU-10           [-1, 14, 10, 10]               0
      BatchNorm2d-11           [-1, 14, 10, 10]              28
           Conv2d-12            [-1, 8, 10, 10]             120
           Conv2d-13             [-1, 12, 8, 8]             876
             ReLU-14             [

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.035105299204587936 Batch_id=468 Accuracy=92.98: 100%|██████████| 469/469 [00:19<00:00, 23.69it/s]



Test set: Average loss: 0.0512, Accuracy: 9844/10000 (98.44%)

EPOCH: 1


Loss=0.031218266114592552 Batch_id=468 Accuracy=98.19: 100%|██████████| 469/469 [00:20<00:00, 22.40it/s]



Test set: Average loss: 0.0379, Accuracy: 9874/10000 (98.74%)

EPOCH: 2


...


Test set: Average loss: 0.0208, Accuracy: 9927/10000 (99.27%)

EPOCH: 13


Loss=0.006811484228819609 Batch_id=468 Accuracy=99.41: 100%|██████████| 469/469 [00:19<00:00, 23.86it/s]



Test set: Average loss: 0.0256, Accuracy: 9913/10000 (99.13%)

EPOCH: 14


Loss=0.07302384078502655 Batch_id=468 Accuracy=99.33: 100%|██████████| 469/469 [00:20<00:00, 22.44it/s]



Test set: Average loss: 0.0235, Accuracy: 9919/10000 (99.19%)



In [18]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_7().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
         MaxPool2d-8           [-1, 12, 12, 12]               0
            Conv2d-9           [-1, 14, 10, 10]           1,526
             ReLU-10           [-1, 14, 10, 10]               0
      BatchNorm2d-11           [-1, 14, 10, 10]              28
           Conv2d-12            [-1, 8, 10, 10]             120
           Conv2d-13             [-1, 16, 8, 8]           1,168
             ReLU-14             [

In [19]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.13829167187213898 Batch_id=468 Accuracy=86.94: 100%|██████████| 469/469 [00:19<00:00, 24.03it/s]



Test set: Average loss: 0.0950, Accuracy: 9703/10000 (97.03%)

EPOCH: 1


Loss=0.03288779780268669 Batch_id=468 Accuracy=97.48: 100%|██████████| 469/469 [00:19<00:00, 24.35it/s]



Test set: Average loss: 0.0608, Accuracy: 9820/10000 (98.20%)

EPOCH: 2


...


Test set: Average loss: 0.0339, Accuracy: 9892/10000 (98.92%)

EPOCH: 13


Loss=0.10614670068025589 Batch_id=468 Accuracy=99.08: 100%|██████████| 469/469 [00:20<00:00, 22.58it/s]



Test set: Average loss: 0.0324, Accuracy: 9900/10000 (99.00%)

EPOCH: 14


Loss=0.02124992199242115 Batch_id=468 Accuracy=99.13: 100%|██████████| 469/469 [00:19<00:00, 24.44it/s]



Test set: Average loss: 0.0309, Accuracy: 9910/10000 (99.10%)



In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_8().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
         MaxPool2d-8           [-1, 12, 12, 12]               0
            Conv2d-9           [-1, 14, 10, 10]           1,526
             ReLU-10           [-1, 14, 10, 10]               0
      BatchNorm2d-11           [-1, 14, 10, 10]              28
           Conv2d-12            [-1, 8, 10, 10]             120
           Conv2d-13           [-1, 12, 10, 10]             876
             ReLU-14           [-1

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.12118884921073914 Batch_id=468 Accuracy=92.91: 100%|██████████| 469/469 [00:19<00:00, 24.52it/s]



Test set: Average loss: 0.0524, Accuracy: 9844/10000 (98.44%)

EPOCH: 1


Loss=0.048466529697179794 Batch_id=468 Accuracy=98.17: 100%|██████████| 469/469 [00:19<00:00, 24.00it/s]



Test set: Average loss: 0.0386, Accuracy: 9888/10000 (98.88%)

EPOCH: 2


...


Test set: Average loss: 0.0222, Accuracy: 9930/10000 (99.30%)

EPOCH: 13


Loss=0.014990908093750477 Batch_id=468 Accuracy=99.35: 100%|██████████| 469/469 [00:19<00:00, 23.90it/s]



Test set: Average loss: 0.0234, Accuracy: 9926/10000 (99.26%)

EPOCH: 14


Loss=0.0385187491774559 Batch_id=468 Accuracy=99.41: 100%|██████████| 469/469 [00:20<00:00, 22.83it/s]



Test set: Average loss: 0.0234, Accuracy: 9929/10000 (99.29%)



In [11]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_9().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
         MaxPool2d-8           [-1, 12, 12, 12]               0
            Conv2d-9           [-1, 14, 10, 10]           1,526
             ReLU-10           [-1, 14, 10, 10]               0
      BatchNorm2d-11           [-1, 14, 10, 10]              28
           Conv2d-12            [-1, 9, 10, 10]             135
           Conv2d-13           [-1, 12, 10, 10]             984
             ReLU-14           [-1

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.04278824105858803 Batch_id=468 Accuracy=93.39: 100%|██████████| 469/469 [00:19<00:00, 23.94it/s]



Test set: Average loss: 0.0510, Accuracy: 9843/10000 (98.43%)

EPOCH: 1


Loss=0.038242604583501816 Batch_id=468 Accuracy=98.39: 100%|██████████| 469/469 [00:19<00:00, 24.09it/s]



Test set: Average loss: 0.0425, Accuracy: 9867/10000 (98.67%)

EPOCH: 2


...


Test set: Average loss: 0.0299, Accuracy: 9899/10000 (98.99%)

EPOCH: 13


Loss=0.003516795113682747 Batch_id=468 Accuracy=99.41: 100%|██████████| 469/469 [00:20<00:00, 22.74it/s]



Test set: Average loss: 0.0220, Accuracy: 9936/10000 (99.36%)

EPOCH: 14


Loss=0.010925475507974625 Batch_id=468 Accuracy=99.41: 100%|██████████| 469/469 [00:20<00:00, 22.79it/s]



Test set: Average loss: 0.0217, Accuracy: 9928/10000 (99.28%)



In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Model_10().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             876
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
         MaxPool2d-8           [-1, 12, 12, 12]               0
            Conv2d-9           [-1, 14, 10, 10]           1,526
             ReLU-10           [-1, 14, 10, 10]               0
      BatchNorm2d-11           [-1, 14, 10, 10]              28
           Conv2d-12            [-1, 9, 10, 10]             135
           Conv2d-13             [-1, 12, 8, 8]             984
             ReLU-14             [

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.0327213816344738 Batch_id=468 Accuracy=90.68: 100%|██████████| 469/469 [00:26<00:00, 17.59it/s]



Test set: Average loss: 0.0584, Accuracy: 9828/10000 (98.28%)

EPOCH: 1


Loss=0.019599488005042076 Batch_id=468 Accuracy=97.80: 100%|██████████| 469/469 [00:26<00:00, 17.67it/s]



Test set: Average loss: 0.0523, Accuracy: 9835/10000 (98.35%)

EPOCH: 2


...


Test set: Average loss: 0.0236, Accuracy: 9926/10000 (99.26%)

EPOCH: 13


Loss=0.0037183088716119528 Batch_id=468 Accuracy=99.14: 100%|██████████| 469/469 [00:27<00:00, 17.34it/s]



Test set: Average loss: 0.0203, Accuracy: 9929/10000 (99.29%)

EPOCH: 14


Loss=0.029377587139606476 Batch_id=468 Accuracy=99.21: 100%|██████████| 469/469 [00:27<00:00, 17.25it/s]



Test set: Average loss: 0.0211, Accuracy: 9932/10000 (99.32%)

